In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import cv2  # importing cv
import find_peak
import scipy.special as special
from matplotlib import ticker
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import nfft
import scipy
import scipy.fft
import functions_analysis as fa
from skimage import img_as_float
from skimage import filters
%matplotlib inline 

path = r'C:\Path\To\Data'
path_main=path+r'\PROCESSED'
scan_number = [ '800nm',  '400nm' ]
scan_name = [ '1.55 eV',  '3.1 eV' ]


path=[]
df_list = []
imgON=[]
imgOFF=[]
delay=[]
# path_upload = path_main+r'\comparison\same_fluence_substraction'
for scan in scan_number:
    path.append(path_main + r'\PROC_'+scan)
    df = pd.read_pickle(path_main + r'\PROC_'+scan, compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})
    imgOFF.append(np.stack(df['imgOFF']))
    imgON.append(np.stack(df['imgON']))
    delay.append(np.stack(df['delay']))
    df_list.append(df)

print(delay)


In [3]:
cc = []
BP_pos = []
BP_name = []
BP_name = ['{100}', '{200}', '{110}']
DS_pos = []
DS_name = [r'$\Gamma \to K$ 1o', r'$\Gamma \to K$ 2o', r'$\Gamma \to M$ 1o', r'$\Gamma \to M$ 2o']
DS_name = [r'$K$', r'$\Gamma \to M$']

Zorder=[]
cal = []
for df in df_list:
    L = df.attrs['peak_dist']
    Zo = df.attrs['Zorder_pos']
    Zorder.append(Zo)
    cal.append(df.attrs['calibration'])
    t0 = df.attrs['t0_index']
    cc.append( L * np.sqrt(3)/3 *np.exp(1j * np.pi  * ( 1./3 * np.arange(7)))) ## for hexagonal BZ
    
    #### construction position of the Bragg peak, ordered by family
    BP_pos.append(fa.bragg_peak_position(Zo, L))

    #### construction position of the K-pont, ordered by family
    DS_pos.append(fa.diffuse_scattering_position2(Zo, L))


In [4]:
for o in range(len(imgON)):
    imgON[o] = filters.gaussian(img_as_float(imgON[o]), sigma=1)
    imgOFF[o] = filters.gaussian(img_as_float(imgOFF[o]), sigma=1)

In [5]:
# plt.rcParams.update({'font.size': 20})
plt.rc('axes', linewidth=2)
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.pad'] = 15
plt.rcParams['ytick.major.pad'] = 15
plt.rcParams['xtick.minor.pad'] = 15
plt.rcParams['ytick.minor.pad'] = 15
plt.rc('axes', linewidth=2)

# plt.rc('font', weight='semibold')
# plt.rcParams['text.latex.preamble'] = r'\usepackage{sfmath} \boldmath'
plt.rc('font', family='arial',size=26)#, weight='bold')
plt.rc('text', usetex=False)

In [ ]:
r_BP = 3
r_DS = 10
count_DS = [] 
count_BP = [] 
for o in range(len(imgON)):
    count=[]
    for i in range(len(BP_pos[o])): 
        count.append(fa.get_count(imgON[o], imgOFF[o], BP_pos[o][i], r_BP))
    count_BP.append(count)
    
    count = []
    for i in range(len(DS_pos[o])):
        count.append(fa.get_count(imgON[o], imgOFF[o], DS_pos[o][i], r_DS, False))
    count_DS.append(count)

print(len(count_BP))

In [ ]:
bt = t0
at = bt
sens = .008
lw = 3
ms=10
lim=7.5
print('area around the Bragg peak: ', np.pi*(2*cal[0])**2)
print('area around the DS point: ', np.pi*(10*cal[0])**2)
fig, axis = plt.subplots(1,3, figsize=(27, 9), width_ratios=[0.48, 0.48, 0.52], layout='tight')


## 1.55 eV
o=0
at = (delay[o] > 0.5) & (delay[o] < 2.5)
bt = (delay[o] < 0)
img = (np.nanmean(imgON[o][at], axis=0))/np.nanmean(imgOFF[o][at], axis=0) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
s1 = np.shape(imgON[o])[2]
s2 = np.shape(imgON[o])[1]
ext = (cal[o]*(Zorder[o][1]), cal[o]*(Zorder[o][1]-s1), cal[o]*(Zorder[o][0]-s2), cal[o]*(Zorder[o][0]))



im1= axis[1].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')
axis[1].set_xlabel(r'$q_x$ $(\AA^{-1})$')
axis[1].set_ylabel(r'$q_y$ $(\AA^{-1})$')
axis[1].set_xlim(-lim,lim)
axis[1].set_ylim(-lim,lim)
axis[1].set_xticks([-5, 0, 5])
axis[1].set_yticks([-5, 0, 5])

t=axis[1].text(4.8, 6.5, scan_name[o], c='k', fontsize = 22, fontweight = 'semibold')
t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))


## 3.1 eV
o=1
at = (delay[o] > 0.5) & (delay[o] < 2.5)
bt = (delay[o] < 0)
img = (np.nanmean(imgON[o][at], axis=0))/np.nanmean(imgOFF[o][at], axis=0) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
s1 = np.shape(imgON[o])[2]
s2 = np.shape(imgON[o])[1]
ext = (cal[o]*(Zorder[o][1]), cal[o]*(Zorder[o][1]-s1), cal[o]*(Zorder[o][0]-s2), cal[o]*(Zorder[o][0]))



im1= axis[2].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')
axis[2].set_xlabel(r'$q_x$ $(\AA^{-1})$')
axis[2].set_ylabel(r'$q_y$ $(\AA^{-1})$')
axis[2].set_xlim(-lim,lim)
axis[2].set_ylim(-lim,lim)
axis[2].set_xticks([-5, 0, 5])
axis[2].set_yticks([-5, 0, 5])
divider = make_axes_locatable(axis[2])
cax = divider.append_axes('right', size='5%', pad=0.2)        
fig.colorbar(im1, cax=cax, shrink=0.6, orientation='vertical', label = r'electron count variation ($\%$)',ticks=[-0.008, 0, 0.008], location='right')
cax.set_yticklabels([-0.8,0,0.8])
t=axis[2].text(5, 6.5, scan_name[o], c='k', fontsize = 22, fontweight = 'semibold')
t.set_bbox(dict(facecolor='white', alpha=0.8, edgecolor='white'))

c_bp = ['k', 'orange', 'g']
c_ds = ['b', 'm', 'y', 'sienna']
s = ['P-', 'o-', '+-']

for i in range(len(BP_pos[o])):
    for j in range(len(BP_pos[o][i])):
        axis[1].add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
               lw=0.2, edgecolor='gray', clip_on=False, facecolor='none'))
        axis[2].add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
               lw=0.2, edgecolor='gray', clip_on=False, facecolor='none'))
         
for i in [0]:
    for j in range(len(BP_pos[o][i])):
        axis[1].add_patch(Circle((cal[o]*(Zorder[o][1]-BP_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-BP_pos[o][i][j][0])), 2*cal[o], fill = False, edgecolor = 'k', lw = 2))
        axis[2].add_patch(Circle((cal[o]*(Zorder[o][1]-BP_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-BP_pos[o][i][j][0])), 2*cal[o], fill = False, edgecolor = 'k', lw = 2))

 
for i in [0,1]:
    for j in range(len(DS_pos[o][i])):
        axis[1].add_patch(Circle((cal[o]*(Zorder[o][1]-DS_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-DS_pos[o][i][j][0])),10*cal[o], fill = False, edgecolor = c_ds[i], lw = 2))
        axis[2].add_patch(Circle((cal[o]*(Zorder[o][1]-DS_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-DS_pos[o][i][j][0])),10*cal[o], fill = False, edgecolor = c_ds[i], lw = 2))



end = len(delay)
leg_bp = []
leg_scan = []
for i in range(len(count_BP)):
    for j in [0]:
        leg_bp.append(axis[0].plot(delay[i], 1e2*(count_BP[i][j]/count_BP[i][j][2]-1), s[i], lw=lw, c=c_bp[j],markersize=ms, label=BP_name[j])[0])
    leg_scan.append(axis[0].plot(delay[i], 1e2*(count_BP[i][0]/count_BP[i][0][2]-1), s[i], lw=lw,c=c_bp[0],markersize=ms, label=scan_number[i])[0])

for i in range(len(count_DS)):
    for j in [0,1]:
        leg_bp.append(axis[0].plot(delay[i], 1e2*(count_DS[i][j]/count_DS[i][j][2]-1), s[i],lw=lw, c=c_ds[j], markersize=ms, label=DS_name[j])[0])


name_point = [r'$\{100\}$', r'$K$', r'$\Gamma\to M$']
leg = [leg_bp[0], leg_bp[2], leg_bp[3]]

leg1 = axis[0].legend(leg, name_point, loc='upper left')
leg2 = axis[0].legend(leg_scan, scan_name, loc='upper right')

axis[0].add_artist(leg1)
axis[0].add_artist(leg2)
axis[0].set_box_aspect(1)

axis[0].set_xlabel(r'delay (ps)')
axis[0].set_ylabel(r'intensity variation ($\%$)')
axis[0].set_xlim(-2, 8)


In [ ]:

#### normalized diffraction map at different time delays ######

bt = t0
at = bt
sens = .008
lw = 3
ms=10
print('area around the Bragg peak: ', np.pi*(2*cal[0])**2)
print('area around the DS point: ', np.pi*(10*cal[0])**2)



fig, axis = plt.subplots(2,4, figsize=(18, 9), layout='tight')


### ------------------------------------------------
o=0
at = 3
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[0][0].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')

o=1
at = 3
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[1][0].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')


### ------------------------------------------------
o=0
at = 12
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[0][1].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')

o=1
at = 12
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[1][1].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')


### ------------------------------------------------
o=0
at = 22
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[0][2].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')

o=1
at = 22
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[1][2].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')



### ------------------------------------------------
o=0
at = 28
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[0][3].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')

o=1
at = 28
bt = (delay[o] < 0)
print('delay = ', delay[o][at], 'for scan : ', o)
img = imgON[o][at]/(imgOFF[o][at]) - (np.nanmean(imgON[o][bt], axis=0))/np.nanmean(imgOFF[o][bt], axis=0)
axis[1][3].imshow(img, cmap='bwr',  extent = ext ,vmin=-sens, vmax=+sens, aspect='equal')

for ax in axis:
    for a in ax:
        a.set_xlim(-lim,lim)
        a.set_ylim(-lim,lim)
        a.set_xticks([])
        a.set_yticks([])
        # a.set_xtickslabel([])
        # a.set_ytickslabel([])
        for i in range(len(BP_pos[o])):
            for j in range(len(BP_pos[o][i])):
                a.add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
                       lw=0.2, edgecolor='gray', clip_on=False, facecolor='none'))
                a.add_patch(Polygon(cal[o]*np.c_[(cc[o].real+Zorder[o][1]-BP_pos[o][i][j][1]), (cc[o].imag+Zorder[o][0]-BP_pos[o][i][j][0])],
                       lw=0.2, edgecolor='gray', clip_on=False, facecolor='none'))
        a.add_patch(Polygon(cal[o]*np.c_[(cc[o].real), (cc[o].imag)],
                       lw=0.2, edgecolor='gray', clip_on=True, facecolor='white'))


for i in [0]:
    for j in range(len(BP_pos[o][i])):
        axis[0][0].add_patch(Circle((cal[o]*(Zorder[o][1]-BP_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-BP_pos[o][i][j][0])), 2*cal[o], fill = False, edgecolor = 'k', lw = 2))
        axis[0][0].add_patch(Circle((cal[o]*(Zorder[o][1]-BP_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-BP_pos[o][i][j][0])), 2*cal[o], fill = False, edgecolor = 'k', lw = 2))

 
for i in [0,1]:
    for j in range(len(DS_pos[o][i])):
        axis[0][0].add_patch(Circle((cal[o]*(Zorder[o][1]-DS_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-DS_pos[o][i][j][0])),10*cal[o], fill = False, edgecolor = c_ds[i], lw = 2))
        axis[0][0].add_patch(Circle((cal[o]*(Zorder[o][1]-DS_pos[o][i][j][1]), cal[o]*(Zorder[o][0]-DS_pos[o][i][j][0])),10*cal[o], fill = False, edgecolor = c_ds[i], lw = 2))
